In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [ ]:
def load_env_and_create_engine():
   # Load environment variables from .env file
    load_dotenv()

    # Get database credentials from environment
    pg_user = os.environ.get('POSTGRES_USER', 'root')
    pg_pass = os.environ.get('POSTGRES_PASSWORD', 'root')
    pg_host = os.environ.get('POSTGRES_HOST', 'localhost')
    pg_port = os.environ.get('POSTGRES_PORT', '5432')
    pg_db = os.environ.get('POSTGRES_DB', 'ny_taxi')
    chunksize = int(os.environ.get('CHUNKSIZE', '100000'))
    #create db postgres  db connection
    engine = create_engine(f'postgresql+psycopg://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}') 
    return engine 


In [ ]:
def run_insert_table():
    #create df iterator
    df_iter = pd.read_parquet(
        'green_tripdata_2025-11.parquet', 
        iterator=True,
        chunksize=chunksize
    )

    #insert df to db
    is_table_head = True
    for df_chunk in tqdm(df_iter):
        #if table head, replace table
        if is_table_head:
            df_chunk.to_sql(name={target_table}, con=engine, if_exists='replace')
            is_table_head = False
        #if not table head, append to table
        else:
            df_chunk.to_sql(name={target_table}, con=engine, if_exists='append')